# rectification

In [ ]:
import glob
import json
import os
from multiprocessing import Pool, pool

import cv2
import matplotlib.pyplot as plt
import numpy as np
import csv

In [ ]:
from scipy.io import loadmat
import tables

In [ ]:
exp2cal = {}
with open('/root/data/small_pen_data_collection/calibration.csv', 'r') as f:
    cr = csv.reader(f)
    for (i, row) in enumerate(cr):
        if i == 0:
            continue
        exp2cal[row[0]] = row[1]

In [ ]:
# load the parameters
def load_params(params_file):
# params = json.load(open('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/stereo_parameters.json'))
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    # return cameraMatrix1, cameraMatrix2, distCoeffs1, distCoeffs2, R, T
    imageSize = (4096, 3000)
    # rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

In [ ]:
rectification_files = glob.glob('/root/data/small_pen_data_collection/underwater_enclosure_0*/parameters*')

In [ ]:
rectification_files

In [ ]:
mapdict = {}
for recfile in rectification_files:
    left_maps, right_maps = load_params(recfile)
    exp = recfile.split('/')[-2]
    mapdict[exp] = {}
    mapdict[exp]['left'] = left_maps
    mapdict[exp]['right'] = right_maps

In [ ]:
mapdict

In [ ]:
# good_sides = ['181008010001', 
#               '181010010001',
#               '181010010002',
#               '181010010003',
#               '181012010001',
#               '181012010002',
#               '181012010003',
#               '181012010004',
#               '181012010005',
#               '181012010007',
#               '181012010008',
#               '181012010009',
#               '181012010010',
#               '181012010011',
#               '181012010012',
#               '181012010013',
#               '181012010014',
#               '181015010001',
#               '181015010002',
#               '181015010003',
#               '181015010004',
#               '181015010005',
#               '181015010006',
#               '181015010007',
#               '181016010001',
#               '181016010002',
#               '181016010003',
#               '181016010004',
#               '181016010005',
#               '181016010006',
#               '181016010007',
#               '180921010001'
#               ]

In [ ]:
# bad_sides = ['181001010001',
#              '181001010002',
#              '181001010003',
#              '181001010004',
#              '181001010005',
#              '181001010006',
#              '181001010007',
#              '181001010008',
#              '181001010009',
#              '181001010010',
#              ]

In [ ]:
# for d in os.listdir('/root/data/small_pen_data_collection/'):
#     if 'underwater' in d:
#         continue
#     if 'rectified' in d:
#         continue
#     if d not in good_sides + bad_sides:
#         print(d)

### ALL FOLDERS

In [ ]:
from collections import Counter

In [ ]:
rectified_experiences = glob.glob('/root/data/small_pen_data_collection/*_rectified')
done = [os.path.basename(p).split('_')[0] for p in rectified_experiences]

In [ ]:
base_dir = '/root/data/small_pen_data_collection/'
experiences = os.listdir(base_dir)
all_image_path = []
for experience in experiences:
    folder_path = os.path.join(base_dir, experience)
    if not os.path.isdir(folder_path):
        continue
    if "rectified" in folder_path:
        continue
    if "reference" in folder_path:
        continue
    if 'underwater' in folder_path:
        continue
    if 'good_pairs' in folder_path:
        continue
    if experience in done:
        continue
    if experience[0] != '1':
        continue
#     if '181017010010' not in folder_path:
#         continue
    print(folder_path)
    all_image_path += glob.glob(folder_path + '/*.jpg')

In [ ]:
all_image_path[0].split('/')

In [ ]:
Counter([path.split('/')[-2] for path in all_image_path])

In [ ]:
# left_paths = sorted(glob.glob('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001/left*'))
# right_paths = sorted(glob.glob('/root/data/rnd/small_pen_data_collection/sotra-small-pen/pen-1/2018-10-01/181001010001/right*'))

In [ ]:
def remap_and_save(img_path):
    exp_number = img_path.split('/')[-2]
    # print(exp_number)
    
    if exp_number not in exp2cal:
        cal  ='underwater_enclosure_04'
    else:
        cal = exp2cal[exp_number]
    # print(cal)
    left_maps = mapdict[cal]['left']
    right_maps = mapdict[cal]['right']
    img = cv2.imread(img_path)
    new_path = img_path.replace(exp_number, exp_number + '_rectified')
    if 'left' in img_path:
        img_remap = cv2.remap(img, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
        # new_path = new_path.replace('right', 'left')
    else:
        img_remap = cv2.remap(img, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
        # new_path = new_path.replace('left', 'right')
    if not os.path.isdir(os.path.dirname(new_path)):
        os.makedirs(os.path.dirname(new_path))
    if not os.path.isfile(new_path):
        print(new_path)
        cv2.imwrite(new_path, img_remap)

In [ ]:
# def remap_and_save(img_path):
#     exp_number = img_path.split('/')[-2]
#     left_maps = mapdict[exp]['left']
#     right_maps = mapdict[exp]['right']
#     print(img_path)
#     img = cv2.imread(img_path)
#     new_path = img_path.replace(exp_number, exp_number + '_rectified')
#     if 'right' in img_path:
#         img_remap = cv2.remap(img, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
#         new_path = new_path.replace('right', 'left')
#     else:
#         img_remap = cv2.remap(img, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
#         new_path = new_path.replace('left', 'right')
#     print(new_path)
#     print("#"*10)
#     if not os.path.isdir(os.path.dirname(new_path)):
#         os.makedirs(os.path.dirname(new_path))
#     cv2.imwrite(new_path, img_remap)

In [ ]:
# multiproceesing
p = Pool(6)

In [ ]:
p.map(remap_and_save, all_image_path)